In [3]:
from sympy import symbols, Matrix, sin, cos, eye, simplify, acos, sqrt, pprint
from sympy import atan2, Function, expand_trig, pi

In [6]:
# SO(3) 指数映射 (旋转向量 -> 旋转矩阵)
def so3_exp(omega):
    """罗德里格斯公式: 将角轴向量 -> SO(3) 旋转矩阵"""
    theta = omega.norm()  # 旋转角度
    if theta == 0:
        return eye(3)

    # 归一化旋转轴
    k = omega / theta

    # 叉乘矩阵
    K = Matrix([[0, -k[2], k[1]], [k[2], 0, -k[0]], [-k[1], k[0], 0]])

    # 罗德里格斯公式
    return eye(3) + sin(theta) * K + (1 - cos(theta)) * (K * K)


# SO(3) 对数映射 (旋转矩阵 -> 旋转向量)
def so3_log(R):
    """旋转矩阵 -> 角轴向量"""
    theta = acos((R.trace() - 1) / 2)  # 旋转角度

    # 处理小角度情况
    if theta < 1e-6:
        return Matrix([0, 0, 0])

    # 计算旋转轴
    axis = Matrix([R[2, 1] - R[1, 2], R[0, 2] - R[2, 0], R[1, 0] - R[0, 1]]) / (2 * sin(theta))

    return theta * axis

# ⊞ 算子: 流形上的广义加法
def boxplus(R, omega):
    """ R ⊞ ω = R * Exp(ω) """
    return R * so3_exp(omega)

# ⊟ 算子: 流形上的广义减法
def boxminus(R2, R1):
    """ R2 ⊟ R1 = Log(R1ᵀ * R2) """
    return so3_log(R1.T * R2)

In [5]:
# ========== 符号计算演示 ==========
print("="*50)
print("SO(3) 上的 boxplus (⊞) 和 boxminus (⊟) 算符演示")
print("="*50)

SO(3) 上的 boxplus (⊞) 和 boxminus (⊟) 算符演示


In [9]:
# 定义符号变量
wx, wy, wz = symbols('ω_x ω_y ω_z')
dx, dy, dz = symbols('δ_x δ_y δ_z')

# 初始旋转向量和增量
omega = Matrix([wx, wy, wz])
delta = Matrix([dx, dy, dz])

# 初始旋转矩阵
R_omega = so3_exp(omega)
print("\n初始旋转矩阵 R(ω):")
pprint(R_omega)

# 使用 ⊞ 算子添加增量
R_updated = boxplus(R_omega, delta)
print("\n应用 ⊞ 算子后的旋转矩阵 R(ω) ⊞ δ:")
pprint(simplify(R_updated))

# 使用 ⊟ 算子计算差异
diff = boxminus(R_updated, R_omega)
print("\n使用 ⊟ 算子计算的差异 (R(ω+δ) ⊟ R(ω)):")
pprint(simplify(diff))

# ========== 数值验证 ==========
print("\n" + "="*50)
print("数值验证")
print("="*50)

# 30度绕Z轴旋转
theta_z = pi/6
R_z = so3_exp(Matrix([0, 0, theta_z]))
print(f"\n绕Z轴旋转30°的矩阵:")
pprint(R_z)

# 45度绕Y轴旋转
theta_y = pi/4
R_y = so3_exp(Matrix([0, theta_y, 0]))
print(f"\n绕Y轴旋转45°的矩阵:")
pprint(R_y)

# 使用 ⊞ 算子组合旋转: 先Z后Y
R_combined = boxplus(R_z, Matrix([0, theta_y, 0]))
print("\n组合旋转 (30°Z ⊞ 45°Y):")
pprint(simplify(R_combined))

# 验证组合旋转的正确性
R_expected = so3_exp(Matrix([0, theta_y, theta_z]))
print("\n直接计算的组合旋转矩阵 (30°Z + 45°Y):")
pprint(simplify(R_expected))

# 使用 ⊟ 算子计算差异
diff_combined = boxminus(R_expected, R_combined)
print("\n差异 (期望 ⊟ 计算):")
pprint(simplify(diff_combined))  # 应为零向量

# ========== 流形约束验证 ==========
print("\n" + "="*50)
print("流形约束验证")
print("="*50)

# 随机旋转
R_rand = so3_exp(Matrix([0.1, 0.2, 0.3]))

# 检查旋转矩阵性质
print("\n旋转矩阵性质检查:")
print(f"行列式: {simplify(R_rand.det())}")  # 应为1
print(f"正交性检查 (RᵀR = I):")
pprint(simplify(R_rand.T * R_rand))

# 应用多次 ⊞ 算子
R1 = boxplus(R_rand, Matrix([0.1, 0, 0]))
R2 = boxplus(R1, Matrix([0, 0.2, 0]))
R3 = boxplus(R2, Matrix([0, 0, 0.3]))

# 检查最终结果是否仍在SO(3)上
print("\n三次更新后的正交性检查 (RᵀR = I):")
pprint(simplify(R3.T * R3))

# ========== 增量恢复验证 ==========
print("\n" + "="*50)
print("增量恢复验证")
print("="*50)

# 原始旋转
R_orig = so3_exp(Matrix([0.3, -0.2, 0.1]))

# 添加增量
delta_test = Matrix([0.15, -0.25, 0.05])
R_modified = boxplus(R_orig, delta_test)

# 使用 ⊟ 恢复增量
delta_recovered = boxminus(R_modified, R_orig)

print("\n原始增量:")
pprint(delta_test)
print("\n恢复的增量:")
pprint(simplify(delta_recovered))


初始旋转矩阵 R(ω):
⎡                                                                              ↪
⎢⎛       ⎛   _________________________⎞⎞ ⎛              2                      ↪
⎢⎜       ⎜  ╱     2        2        2 ⎟⎟ ⎜           ω_y                       ↪
⎢⎝1 - cos⎝╲╱  │ωₓ│  + │ω_y│  + │ω_z│  ⎠⎠⋅⎜- ─────────────────────── - ──────── ↪
⎢                                        ⎜      2        2        2       2    ↪
⎢                                        ⎝  │ωₓ│  + │ω_y│  + │ω_z│    │ωₓ│  +  ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢            ⎛       ⎛   _________________________⎞⎞          ⎛   ____________ ↪
⎢            ⎜       ⎜  ╱     2        2        2 ⎟⎟          ⎜  ╱     2       ↪
⎢     ωₓ⋅ω_y⋅⎝1 - cos⎝╲╱  │ωₓ│  + │ω_y│  + │ω_z│  ⎠⎠   ω_z⋅sin⎝╲╱  │ωₓ│  + │ω_ ↪
⎢     ────────────────────────────────────────────── + ─────────────────────── ↪
⎢             

TypeError: cannot determine truth value of Relational: acos(((1 - cos(sqrt(Abs(ω_x)**2 + Abs(ω_y)**2 + Abs(ω_z)**2)))*(-ω_x**2/(Abs(ω_x)**2 + Abs(ω_y)**2 + Abs(ω_z)**2) - ω_y**2/(Abs(ω_x)**2 + Abs(ω_y)**2 + Abs(ω_z)**2)) + 1)*(((1 - cos(sqrt(Abs(δ_x)**2 + Abs(δ_y)**2 + Abs(δ_z)**2)))*(-δ_x**2/(Abs(δ_x)**2 + Abs(δ_y)**2 + Abs(δ_z)**2) - δ_y**2/(Abs(δ_x)**2 + Abs(δ_y)**2 + Abs(δ_z)**2)) + 1)*((1 - cos(sqrt(Abs(ω_x)**2 + Abs(ω_y)**2 + Abs(ω_z)**2)))*(-ω_x**2/(Abs(ω_x)**2 + Abs(ω_y)**2 + Abs(ω_z)**2) - ω_y**2/(Abs(ω_x)**2 + Abs(ω_y)**2 + Abs(ω_z)**2)) + 1) + (-δ_x*sin(sqrt(Abs(δ_x)**2 + Abs(δ_y)**2 + Abs(δ_z)**2))/sqrt(Abs(δ_x)**2 + Abs(δ_y)**2 + Abs(δ_z)**2) + δ_y*δ_z*(1 - cos(sqrt(Abs(δ_x)**2 + Abs(δ_y)**2 + Abs(δ_z)**2)))/(Abs(δ_x)**2 + Abs(δ_y)**2 + Abs(δ_z)**2))*(ω_x*sin(sqrt(Abs(ω_x)**2 + Abs(ω_y)**2 + Abs(ω_z)**2))/sqrt(Abs(ω_x)**2 + Abs(ω_y)**2 + Abs(ω_z)**2) + ω_y*ω_z*(1 - cos(sqrt(Abs(ω_x)**2 + Abs(ω_y)**2 + Abs(ω_z)**2)))/(Abs(ω_x)**2 + Abs(ω_y)**2 + Abs(ω_z)**2)) + (δ_x*δ_z*(1 